In [116]:
from datetime import datetime, date
import pandas as pd
import pyspark
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DateType



In [8]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 20:32:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
],  schema = 'a long, b double, c string, d date, e timestamp'

)
print(df)

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]


In [10]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [12]:
df.show(0)

+---+---+---+---+---+
|  a|  b|  c|  d|  e|
+---+---+---+---+---+
+---+---+---+---+---+
only showing top 0 rows



In [14]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
4,5.0,string3,2000-03-01,2000-01-03 12:00:00


In [15]:
df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
only showing top 1 row



In [18]:
df.columns
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [19]:
df.select("a","b","c").describe().show()

+-------+------------------+------------------+-------+
|summary|                 a|                 b|      c|
+-------+------------------+------------------+-------+
|  count|                 3|                 3|      3|
|   mean|2.3333333333333335|3.3333333333333335|   null|
| stddev|1.5275252316519468|1.5275252316519468|   null|
|    min|                 1|               2.0|string1|
|    max|                 4|               5.0|string3|
+-------+------------------+------------------+-------+



In [21]:
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
4,5.0,string3,2000-03-01,2000-01-03 12:00:00


In [24]:
df.filter(df.a ==1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [25]:
df_gro = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df_gro.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



# First real live project 
ETL Extract Transform Load

In [2]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DateType

# Spark The Definitive Guide
Starten einer 'sparkSession'.The SparkSession instance is the way Spark executes user-defined manipulations across the cluster.
Code: 
- Input
    spark = SparkSession.builder.getOrCreate()

Output:
- SparkSession - in-memory

    SparkContext

    Spark UI

    Version
    v3.4.1
    Master
    local[*]
    AppName
    pyspark-shell



###### Start Seite 23


In [32]:
# Load trainsched.txt
spark = SparkSession.builder.getOrCreate()
spark

Hier kann die Sparksession weiter detailiert betrachtet werden.

http://localhost:4040

#### Erstellen eine Spalte von 1000 Reihen Zahlen.
Also 0 - 1.000
Es wird ein Dataframe erstellt. 
Dieser Zahlenbereich stellt eine verteilte Sammlung dar. Wenn er in einem Cluster ausgeführt wird, existiert jeder Teil dieses Zahlenbereichs auf einem anderen Executor. Dies ist ein Spark DataFrame. 
- S.24 Mitte

#### DataFrames
- Ein DataFrame ist die gängigste strukturierte API und stellt einfach eine Datentabelle mit Zeilen und Spalten dar. Die Liste, die die Spalten und die Typen innerhalb dieser Spalten definiert, wird als Schema bezeichnet. 
Sie können sich einen DataFrame wie eine Tabellenkalkulation mit benannten Spalten vorstellen. 

- Eine Tabellenkalkulation befindet sich auf einem Computer an einem bestimmten Ort, während ein Spark DataFrame Tausende von Computern umfassen kann. Der Grund für die Speicherung der Daten auf mehr als einem Computer sollte intuitiv sein: Entweder sind die Daten zu groß, um auf einen Computer zu passen, oder es würde einfach zu lange dauern, diese Berechnung auf einem Computer durchzuführen.

- Spark verfügt über mehrere zentrale Abstraktionen: Datensätze, DataFrames, SQL-Tabellen und belastbare verteilte Datensätze (RDDs). Diese verschiedenen Abstraktionen stellen alle verteilte Datensammlungen dar. Am einfachsten und effizientesten sind DataFrames, die in allen Sprachen verfügbar sind. Wir behandeln Datasets am Ende von Teil II, und RDDs


In [33]:
myRange = spark.range(1000).toDF("number")


In [35]:
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



### Partions 

- Damit jeder Executor die Arbeit parallel ausführen kann, unterteilt Spark die Daten in Stücke, die Partitionen genannt werden. 
Eine Partition ist eine Sammlung von Zeilen, die sich auf einer physischen Maschine in Ihrem Cluster befinden. Die Partitionen eines DataFrame stellen dar, wie die Daten während der Ausführung physisch über den Rechnercluster verteilt werden. Wenn Sie eine Partition haben, hat Spark nur eine Parallelität, auch wenn Sie Tausende von Executors haben. Wenn Sie viele Partitionen, aber nur einen Executor haben, hat Spark immer noch eine Parallelität von nur einer, da es nur eine Berechnungsressource gibt.

--> (Partions oder auch Parallelität bedeutet wie viele Executor parallel auf den Dataset arbeiten können. Als Team wünsch man sich eine hohe Parallelität)>

--> Im Kontext von Datenbanken können Partitionen verwendet werden, um eine große Tabelle in kleinere, handhabbare Segmente aufzuteilen. Diese Segmente können auf verschiedene Arten organisiert werden, z. B. nach bestimmten Werten in einer Spalte (z. B. nach Datum, Region oder Kundengruppe) oder nach bestimmten Kriterien (z. B. durch Zufallsverteilung).

- Ein wichtiger Punkt ist, dass Sie mit DataFrames (in den meisten Fällen) Partitionen nicht manuell oder einzeln bearbeiten. Sie legen lediglich High-Level-Transformationen von Daten in den physischen Partitionen fest, und Spark bestimmt, wie diese Arbeit tatsächlich im Cluster ausgeführt wird. Es gibt APIs auf niedrigerer Ebene (über die RDD-Schnittstelle), die wir in Teil III behandeln.

### Transformations 

In Spark sind die Kerndatenstrukturen unveränderlich, d. h. sie können nach ihrer Erstellung nicht mehr geändert werden. Dies mag auf den ersten Blick wie ein seltsames Konzept erscheinen: Wenn man sie nicht ändern kann, wie soll man sie dann verwenden? Um einen DataFrame zu "ändern", müssen Sie Spark mitteilen, wie Sie ihn ändern möchten, damit er das tut, was Sie wollen. Diese Anweisungen werden als Transformationen bezeichnet. Lassen Sie uns eine einfache Transformation durchführen, um alle geraden Zahlen in unserem aktuellen DataFrame zu finden:

Zeile 36 führt aus das wir nur grade Zahlen bekommen jedoch wurde damit nicht der Kern datensatz verändert.
Durch Zuweisung in eine andere Variable "divisBy2" haben wir den Kern immer noch nicht verändert, sondern die kurzzeitigen veränderten Datensatz in einem neue Variable

=> Auch Transformation genannt

In [36]:
divisBy2 = myRange.where("number % 2 = 0")


In [38]:
divisBy2.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows




- Beachten Sie, dass diese keine Ausgabe liefern. Dies liegt daran, dass wir nur eine abstrakte Transformation angegeben haben und Spark erst dann auf Transformationen reagiert, wenn wir eine Aktion aufrufen (dazu kommen wir gleich). Transformationen sind der Kern Ihrer Geschäftslogik in Spark. Es gibt zwei Arten von Transformationen: solche, die enge Abhängigkeiten angeben, und solche, die weite Abhängigkeiten angeben.




#### Narrow Transformation 
- Wenn jede Partition im übergeordneten RDD von höchstens einer Partition des untergeordneten RDD verwendet wird, liegt eine enge Abhängigkeit vor. Berechnungen von Transformationen mit dieser Art von Abhängigkeit sind recht schnell, da sie keine Datenumlagerung über das Clusternetzwerk erfordern. Darüber hinaus sind auch Optimierungen wie Pipelining möglich.

#### Wide Transformation
Bei einer Transformation im Stil einer breiten Abhängigkeit (oder breiten Transformation) tragen Eingabepartitionen zu vielen Ausgabepartitionen bei. Dies wird oft als "Shuffle" bezeichnet, wobei Spark Partitionen im gesamten Cluster austauscht. Bei schmalen Transformationen führt Spark automatisch eine Operation durch, die als Pipelining bezeichnet wird, d. h., wenn wir mehrere Filter auf DataFrames angeben, werden sie alle im Speicher ausgeführt. Das Gleiche gilt nicht für Shuffles. Wenn wir einen Shuffle durchführen, schreibt Spark die Ergebnisse auf die Festplatte.

#### Lazy Evaluation

Lazy Evaluation bedeutet, dass Spark bis zum letzten Moment wartet, um den Graphen der Berechnungsanweisungen auszuführen. Anstatt die Daten sofort zu ändern, wenn Sie eine Operation ausführen, erstellen Sie in Spark einen Plan mit Transformationen, die Sie auf Ihre Quelldaten anwenden möchten. Wenn Sie mit der Ausführung des Codes bis zur letzten Minute warten, kompiliert Spark diesen Plan aus Ihren rohen DataFrame-Transformationen zu einem rationalisierten physischen Plan, der so effizient wie möglich im Cluster ausgeführt wird. Dies bietet immense Vorteile, da Spark den gesamten
 
gesamten Datenfluss von Ende zu Ende optimieren kann. Ein Beispiel hierfür ist das so genannte Prädikat Pushdown auf DataFrames. Wenn wir einen großen Spark-Job erstellen, aber am Ende einen Filter angeben, der nur eine Zeile aus unseren Quelldaten abrufen muss, ist der effizienteste Weg zur Ausführung der Zugriff auf den einzelnen benötigten Datensatz. Spark optimiert diesen Vorgang für uns, indem es den Filter automatisch nach unten verschiebt.

In [41]:
"The Frist df:", myRange.count(), "The Second df:", divisBy2.count()

('The Frist df:', 1000, 'The Second df:', 500)

In [43]:
flightData2015 = spark.read.option("inferSchema", 
                                   'true').option("header", 
                                                  "true").csv("/Users/riccardo/Desktop/Repositorys_Github/Training/Dataset/2015-summary.csv")


Remember, sort does not modify the DataFrame. We use sort as a transformation that returns a new DataFrame by transforming the previous DataFrame. Let’s illustrate what’s happening when we call take on that resulting DataFrame

In [44]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [45]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#3767 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#3767 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=452]
      +- FileScan csv [DEST_COUNTRY_NAME#3765,ORIGIN_COUNTRY_NAME#3766,count#3767] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/riccardo/Desktop/Repositorys_Github/Training/Dataset/2015-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




Wie zuvor können wir nun eine Aktion festlegen, um diesen Plan zu starten. Bevor wir das tun, müssen wir jedoch noch eine Konfiguration festlegen. Wenn wir einen Shuffle durchführen, gibt Spark standardmäßig 200 Shuffle-Partitionen aus. Wir setzen diesen Wert auf 5, um die Anzahl der ausgegebenen Partitionen des Shuffle zu reduzieren:
  spark.conf.set("spark.sql.shuffle.partitions", "5")
  flightData2015.sort("count").take(2)
  ... Array([Vereinigte Staaten,Singapur,1], [Moldawien,Vereinigte Staaten,1])
  
Abbildung 2-9 veranschaulicht diesen Vorgang. Beachten Sie, dass zusätzlich zu den logischen Transformationen auch die Anzahl der physischen Partitionen berücksichtigt wird.

csv File ->           Dataframe ->            -> Dataframe ->     Array()
       Read (narrow)              sort (wide)             take(3)
                      1 Partion                   5 Partion

In [47]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [48]:
flightData2015.createGlobalTempView("flightData2015")

In [4]:
txt_file_path = "/Users/riccardo/Desktop/Repositorys_Github/Training/Dataset/produkt_klima_tag_19820101_20221231_01766.txt"

spark_df = spark.read.option("sep", ";").option("header", "true").csv(txt_file_path)

# Create temporary table called table1
spark_df.createOrReplaceTempView("table1")
spark_df.show()


+-----------+----------+-----+------+------+----+----+----+----+-------+----+----+----+----+----+----+----+----+---+
|STATIONS_ID|MESS_DATUM| QN_3|    FX|    FM|QN_4| RSK|RSKF| SDK|SHK_TAG|  NM| VPM|  PM| TMK| UPM| TXK| TNK| TGK|eor|
+-----------+----------+-----+------+------+----+----+----+----+-------+----+----+----+----+----+----+----+----+---+
|       1766|  19820101|   10|   6.7|   2.4|-999|-999|-999|-999|   -999|-999|-999|-999|-999|-999|-999|-999|-999|eor|
|       1766|  19820102|   10|  10.3|   4.5|-999|-999|-999|-999|   -999|-999|-999|-999|-999|-999|-999|-999|-999|eor|
|       1766|  19820103|   10|  13.9|   5.6|-999|-999|-999|-999|   -999|-999|-999|-999|-999|-999|-999|-999|-999|eor|
|       1766|  19820104|   10|  18.0|   7.1|-999|-999|-999|-999|   -999|-999|-999|-999|-999|-999|-999|-999|-999|eor|
|       1766|  19820105|   10|  14.4|   6.5|-999|-999|-999|-999|   -999|-999|-999|-999|-999|-999|-999|-999|-999|eor|
|       1766|  19820106|   10|  17.5|   7.3|-999|-999|-999|-999|

In [31]:
df_2 = spark_df.filter((spark_df.RSKF != -999) &
                       (spark_df.MESS_DATUM >= 19891001) & 
                       (spark_df.MESS_DATUM <= 19891010))
df_2.show()

+-----------+----------+----+----+----+----+----+----+----+-------+----+----+----+----+----+----+----+----+----+
|STATIONS_ID|MESS_DATUM|QN_3|  FX|  FM|QN_4| RSK|RSKF| SDK|SHK_TAG|  NM| VPM|  PM| TMK| UPM| TXK| TNK| TGK| eor|
+-----------+----------+----+----+----+----+----+----+----+-------+----+----+----+----+----+----+----+----+----+
|       1766|  19891001|  10|   9|   3|  10|   0|   6|   0|      0|   7|  13|1020|  12|  92|  15|  10|  10|null|
|       1766|  19891002|  10|  12|   4|  10|   0|   6|   2|      0|   7|  11|1016|  12|  77|  15|  10|  10|null|
|       1766|  19891003|  10|   9|   2|  10|   0|   0|  10|      0|   1|   9|1018|   8|  77|  15|   4|   8|null|
|       1766|  19891004|  10|   3|   0|  10|   0|   0|  10|      0|   0|   7|1021|   6|  77|  15|  -1|  -2|null|
|       1766|  19891005|  10|   7|   2|  10|   0|   6|  10|      0|   1|   9|1016|  11|  70|  19|   3|   1|null|
|       1766|  19891006|  10|  11|   4|  10|  11|   6|   3|      0|   7|  11|1009|  11|  83|  15

In [5]:
spark_df.describe().show()
spark_df.printSchema()
spark_df.columns

24/02/24 12:40:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------+-------------------+------------------+-------------------+------------------+------------------+-----+
|summary|STATIONS_ID|          MESS_DATUM|              QN_3|                FX|                FM|              QN_4|               RSK|               RSKF|                SDK|            SHK_TAG|                 NM|                VPM|               PM|                TMK|               UPM|                TXK|               TNK|               TGK|  eor|
+-------+-----------+--------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------+-------------------+-------

['STATIONS_ID',
 'MESS_DATUM',
 'QN_3',
 '  FX',
 '  FM',
 'QN_4',
 ' RSK',
 'RSKF',
 ' SDK',
 'SHK_TAG',
 '  NM',
 ' VPM',
 '  PM',
 ' TMK',
 ' UPM',
 ' TXK',
 ' TNK',
 ' TGK',
 'eor']

In [12]:
spalten = spark_df.columns
for spalte in spalten:
    spark_df.withColumn(spalte, col(spalte).cast("integer"))

# DataFrame anzeigen
spark_df.printSchema()

root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: integer (nullable = true)
 |-- QN_3: integer (nullable = true)
 |--   FX: integer (nullable = true)
 |--   FM: integer (nullable = true)
 |-- QN_4: integer (nullable = true)
 |--  RSK: integer (nullable = true)
 |-- RSKF: integer (nullable = true)
 |--  SDK: integer (nullable = true)
 |-- SHK_TAG: integer (nullable = true)
 |--   NM: integer (nullable = true)
 |--  VPM: integer (nullable = true)
 |--   PM: integer (nullable = true)
 |--  TMK: integer (nullable = true)
 |--  UPM: integer (nullable = true)
 |--  TXK: integer (nullable = true)
 |--  TNK: integer (nullable = true)
 |--  TGK: integer (nullable = true)
 |-- eor: integer (nullable = true)



In [13]:
result_df = spark.sql("SELECT * FROM table1")
result_df.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |-- QN_3: string (nullable = true)
 |--   FX: string (nullable = true)
 |--   FM: string (nullable = true)
 |-- QN_4: string (nullable = true)
 |--  RSK: string (nullable = true)
 |-- RSKF: string (nullable = true)
 |--  SDK: string (nullable = true)
 |-- SHK_TAG: string (nullable = true)
 |--   NM: string (nullable = true)
 |--  VPM: string (nullable = true)
 |--   PM: string (nullable = true)
 |--  TMK: string (nullable = true)
 |--  UPM: string (nullable = true)
 |--  TXK: string (nullable = true)
 |--  TNK: string (nullable = true)
 |--  TGK: string (nullable = true)
 |-- eor: string (nullable = true)

